<a href="https://colab.research.google.com/github/eireford/ConwayLifeTensorFlow/blob/master/ConwayLifeTensorFlow_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
import tensorflow as tf

In [0]:
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.set_printoptions(precision=4)

While the rules of the game of life can be simplified, we are going to experiment using the rules that are othagonal to Conways conceptualization.
1. Any live cell with fewer than two live neighbours dies, as if by underpopulation.
2. Any live cell with two or three live neighbours lives on to the next generation.
3. Any live cell with more than three live neighbours dies, as if by overpopulation.
4. Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

For these rules we are going to denormalize 
 survive_min, survive_max, reproduce_min, reproduce_max

In [0]:
life_density_kernal = tf.constant([1,1,1,1,10,1,1,1,1],dtype=tf.float32)
data_set_size = tf.constant([30],dtype=tf.int32)
sustainable_mean = tf.constant([12.5],dtype=tf.float32)
sustainable_range = tf.constant([2],dtype=tf.float32)
productive_mean = tf.constant([3],dtype=tf.float32)
productive_range = tf.constant([1],dtype=tf.float32)

In [0]:
def sustainable(x_sum): 
  lower_bound = sustainable_mean - (sustainable_range/2)
  upper_bound = sustainable_mean + (sustainable_range/2)
  if lower_bound < x_sum and x_sum < upper_bound:
    return tf.constant([1],dtype=tf.float32)
  else: return tf.constant([0],dtype=tf.float32)
def productive(x_sum):
  lower_bound = productive_mean - (productive_range/2)
  upper_bound = productive_mean + (productive_range/2)
  if lower_bound < x_sum and x_sum < upper_bound:
    return tf.constant([1],dtype=tf.float32)
  else: return tf.constant([0],dtype=tf.float32)
  return lower_bound < x_sum and x_sum < upper_bound
def sustainable_and_productive(x):
  x_sum = tf.reduce_sum(x)
  is_sustainable = sustainable(x_sum)
  is_productive = productive(x_sum)
  result = tf.concat([is_sustainable,is_productive],axis=0,name="label")
  return result

Create a random dataset as simply as possible.

In [0]:
def get_data_examples(count):
  for i in range(count*2):
    data = tf.round(tf.random.uniform([life_density_kernal.shape[0]],minval=0,maxval=1,dtype=tf.float32),name="data")
    #print(data)
    data_sum = tf.reduce_sum(data)
    #print(data_sum)
    label = sustainable_and_productive(data_sum)
    result = (data,label)
    yield result

In [86]:
def a_typical_run():
  data_gen = get_data_examples(100_000)
  for data in data_gen:
    element = next(data_gen)

%timeit a_typical_run()

1 loop, best of 3: 1min 38s per loop
